# Optimize under simulation
Since simulator will generate 1 list of pool history and we need to optimize the sum of all apy from each pool, we will use this notebook to examine the effect of different optimize algorithm.


In [63]:
%load_ext autoreload
%autoreload 2
import copy
import json
from sturdy.pools import generate_assets_and_pools
from sturdy.protocol import AllocateAssets
from sturdy.utils.sim_yiop import sim_yiop_algorithm, simulated_yiop_allocation_algorithm_2, global_yiop_allocation_algorithm, simulated_yiop_allocation_algorithm
from sturdy.utils.yiop import yiop_allocation_algorithm
from sturdy.validator import simulator
from sturdy.validator.reward import calculate_aggregate_apy

allocations = {'0': 1, '1': 0}

def load_assets_and_pools(filename):
    with open(filename, 'r') as f:
        return json.load(f)
assets_and_pools = load_assets_and_pools('../sample_pools_and_miner_answers.json')
assets_and_pools['total_assets'] = 2.0
# print(assets_and_pools['answers']['136'])

new_ap = generate_assets_and_pools()
assets_and_pools['pools'] = new_ap['pools']
allocation = assets_and_pools['answers']['136']['allocations']

def get_sim(allocation):
    sim = simulator.Simulator()
    sim.initialize()
    sim.reset()
    # sim_allocation = {k: 0 for Vk in allocation.keys()}
    sim_allocation = allocation
    sim.init_data(init_assets_and_pools=copy.deepcopy(assets_and_pools), init_allocations=sim_allocation)
    sim.update_reserves_with_allocs()
    sim.run()
    return sim

sim = simulator.Simulator()
sim.initialize()
sim.reset()
sim.init_data(init_assets_and_pools=copy.deepcopy(assets_and_pools), init_allocations=allocation)
sim.update_reserves_with_allocs()
sim.run()

yiop_allocation = yiop_allocation_algorithm(AllocateAssets(assets_and_pools=assets_and_pools))
new_allocation = simulated_yiop_allocation_algorithm(AllocateAssets(assets_and_pools=assets_and_pools))

sim = get_sim(allocation)
print('best', calculate_aggregate_apy(allocation, assets_and_pools, sim.timesteps, sim.pool_history))
sim = get_sim(yiop_allocation)
print('yiop', calculate_aggregate_apy(yiop_allocation, assets_and_pools, sim.timesteps, sim.pool_history))
sim = get_sim(new_allocation)
print('new', calculate_aggregate_apy(new_allocation, assets_and_pools, sim.timesteps, sim.pool_history))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
best 15.397214925855852
yiop 16.524395009716375
new 16.735838130315905
